In [ ]:
import os, sys
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
import json
import re

import nltk
import evaluate
from transformers import DataCollatorForSeq2Seq, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
from transformers import BertTokenizer, AutoModel

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from repo.indobenchmark.toolkit.tokenization_indonlg import IndoNLGTokenizer

/Users/siagian/workspace/thesis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# check if cuda or mps available, if available, use one of them, otherwise use cpu

device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = (
        "1"  # This is tracked as pytorch issue #98222
    )
    print("using mps")
else:
    device = torch.device("cpu")
    print("using cpu")


using mps


### Data Loading

In [7]:
ds = load_dataset("./repo/SEACrowd/indosum/indosum.py")

# check the length of the dataset
print("Train dataset length: ", len(ds["train"]))
print("Validation dataset length: ", len(ds["validation"]))
print("Test dataset length: ", len(ds["test"]))
print("")

# explore the first 5 data in the dataset
print(json.dumps(ds["train"][:5], indent=4))
print("")

Train dataset length:  14262
Validation dataset length:  750
Test dataset length:  3762

{
    "document": [
        "Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pada Jumat (4 / 8) dini hari. Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu. Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia. Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru, Riau untuk menjalani istirahat. \" Setahu saya dia orangnya sehat, tapi tahun lalu saya dengar dia sakit. (Karena) sakitnya, ia langsung pulang ke Pekanbaru, jadi kami yang mau jenguk juga susah. Barangkali mau istirahat, ya betul juga, kalau di Jakarta susah isirahatnya, \" kata Lula kepada CNNIndonesia.com, Jumat (4 / 8). Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sem

#### Dataset Pre-Processing

Affected dataset splits: 
- train
- validation

We will perform the following pre-processing steps:
1. case folding
<!-- 2. remove special characters -->
1. remove stopwords (Bahasa Indonesia)


In [ ]:
# Lowercase the text
ds["train"] = ds["train"].map(lambda x: {"document": x["document"].lower(), "summary": x["summary"].lower()})
ds["validation"] = ds["validation"].map(lambda x: {"document": x["document"].lower(), "summary": x["summary"].lower()})

# remove special characters

# remove stopwords
stopwords = StopWordRemoverFactory().get_stop_words()
ds["train"] = ds["train"].map(lambda x: {"document": ' '.join([word for word in x["document"].split() if word not in stopwords])})

print(json.dumps(ds["train"][:5], indent=4))

{
    "document": [
        "jakarta, cnn indonesia - - dokter ryan thamrin, yang terkenal lewat acara dokter oz indonesia, meninggal dunia pada jumat (4 / 8) dini hari. dokter lula kamal yang merupakan selebriti sekaligus rekan kerja ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu. lula menuturkan, sakit itu membuat ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara dokter oz indonesia. kondisi itu membuat ryan harus kembali ke kampung halamannya di pekanbaru, riau untuk menjalani istirahat. \" setahu saya dia orangnya sehat, tapi tahun lalu saya dengar dia sakit. (karena) sakitnya, ia langsung pulang ke pekanbaru, jadi kami yang mau jenguk juga susah. barangkali mau istirahat, ya betul juga, kalau di jakarta susah isirahatnya, \" kata lula kepada cnnindonesia.com, jumat (4 / 8). lula yang mengenal ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk ryan lantaran lokasi yang jauh. dia juga tak tahu penyakit apa yang diderita